In [1]:
import numpy as np
import math


In [3]:
L, d_k,d_v = 4,8,8

q = np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_v)

print(f'q: {q} \n {q.shape}')

q: [[-0.64160862  0.30220004 -0.61595845  1.20250417 -0.18510138  0.33408615
  -0.66467492 -1.56618377]
 [-0.05870135  1.47366876  0.47503371 -1.29730791 -1.73193787 -0.95141771
   0.68898643 -0.18032509]
 [-0.1328231   0.2566022  -1.35322708 -0.56658146  0.26455423 -0.81648327
  -0.51199825  0.3024339 ]
 [ 0.03559231 -0.04839526  1.56351574  0.4234759   0.52231569 -0.66089308
   0.61332584  0.90518989]] 
 (4, 8)


Self Attention

$self attention = softmax{({QK^T \over \sqrt{d_k}} +M)V}$

In [6]:
np.matmul(q,k.T)
# same as:
q@k.T

array([[-1.23973136,  2.42082008,  2.6021219 , -1.24145812],
       [ 4.47763001,  1.53169749, -1.26457367, -2.29504025],
       [ 1.6052039 ,  1.61075147, -1.91492468, -0.65375085],
       [-2.40674416, -0.73653602, -1.69170908,  3.42647261]])

In [11]:
scaled_q_k = q@k.T / np.sqrt(d_k)
scaled_q_k

array([[-0.43831122,  0.85588915,  0.91998902, -0.43892173],
       [ 1.58308127,  0.54153684, -0.44709431, -0.81141926],
       [ 0.56752528,  0.56948664, -0.67702811, -0.23113583],
       [-0.85091256, -0.26040481, -0.59810948,  1.21144101]])

In [12]:
q.var(),k.var(),scaled_q_k.var()

(0.6721539920638319, 1.2456821566762712, 0.57856078371441)

In [7]:
#masking
# lets words only look at tokens that came before it
mask = np.tril(np.ones((L,L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [9]:
mask[mask==0] = -np.infty
mask[mask==1] = 0
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [13]:
scaled_q_k + mask

array([[-0.43831122,        -inf,        -inf,        -inf],
       [ 1.58308127,  0.54153684,        -inf,        -inf],
       [ 0.56752528,  0.56948664, -0.67702811,        -inf],
       [-0.85091256, -0.26040481, -0.59810948,  1.21144101]])

In [14]:
#softmax converts vector into prob distribution
def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T



In [15]:
attention = softmax(scaled_q_k + mask)
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.7391479 , 0.2608521 , 0.        , 0.        ],
       [0.43667496, 0.43753228, 0.12579275, 0.        ],
       [0.08363307, 0.15094972, 0.10768843, 0.65772877]])

In [16]:
new_v = attention @ v
new_v

array([[-0.4725925 ,  0.81298057, -1.43956008,  0.21747511,  2.15076971,
        -0.05137905,  1.10195603, -1.07178402],
       [-0.09684792,  0.87059508, -0.96400721, -0.3346464 ,  1.52854287,
        -0.04017561,  0.7505509 , -1.23124724],
       [ 0.16778739,  0.79517111, -0.25758252, -0.61963966,  0.78613471,
        -0.0046214 ,  0.20529551, -1.02307423],
       [ 0.32832328,  0.28124412,  0.7735031 , -0.31600945,  0.55560734,
         0.996375  , -0.44048683,  0.05990275]])

When $a \ne 0$, there are two solutions to $(ax^2 + bx + c = 0)$ and they are 
$$ x = {-b \pm \sqrt{b^2-4ac} \over 2a} $$